<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>    

# Cómo trabajar con datasets

En este notebook vamos a ver cómo podemos cargar un dataset almacenado en un csv. Para ello utilizaremos la librería Pandas. 

## Montar nuestro disco de google drive en Google Colab.

Para acceder al fichero csv de nuestro dataset, primero tendremos que montar el disco de google drive. La siguiente celda contiene el código necesario para realizar dicha operación. Como paso necesario, se solicita el permiso para poder acceder a los archivos de google drive.

In [1]:
from google.colab import drive
# mount your google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Vamos a definir una variable que almacene la ruta al directorio en google drive donde almacenamos los datasets:

In [2]:
PATH = "/content/drive/My Drive/Colab Notebooks/data/"

Cada vez existen más datasets disponibles para tareas de PLN. Por ejemplo, [este página de Kaggle](#https://www.kaggle.com/datasets?search=text+classification) contiene una lista de los datasets más populares para la tarea de clasificación de textos. 
En este caso, vamos a utilizar un dataset que consiste en una colección de tweets sobre coronavirus anotados para la tarea de sentiment analysis (detectar el sentimiento de un texto: positivo,  negativo o neutro). Para descargar este dataset, visita el siguiente [link](#https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification?resource=download).

El dataset contiene dos ficheros csv distintos, uno para la fase de entrenamiento y el segundo para la fase de evaluación. 

En la siguiente celda, utilizamos la librería de Pandas para cargar el fichero 'Corona_NLP_train.csv'.

In [3]:
import pandas as pd

df_train = pd.read_csv(PATH+"corona/Corona_NLP_train.csv", encoding='latin-1') 
print('training dataset was loaded ', df_train.shape)

#show the first rows
df_train.head()

training dataset was loaded  (41157, 6)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


Podemos ver que cada registro del dataset tiene los siguientes campos: UserName, ScreenName, Location, TweetAt, OriginalTweet, Sentiment.

En realidad, para la tarea de análisis de sentimiento simplemente necesitamos trabajar con el texto del tweet y con su sentimiento correspondiente. Podemos ignorar el resto de los campos.

In [4]:
df_train = df_train[['OriginalTweet','Sentiment']] # we ignore the other columns
df_train.head()


,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


Ahora vamos a renombrar el campo 'OriginalTweet' a 'Text', y el campo 'Sentiment' a 'Label':

In [5]:
df_train=df_train.rename(columns={"OriginalTweet": "Text", "Sentiment": "Label"})
df_train.head()

,Text,Label
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


### Ejercicio:
Carga el fichero 'Corona_NLP_test.csv', visualiza los primeros registros. 
Después, al igual que hemos hecho con el dataset de training, únicamente queremos trabajar con los campos 'OriginalTweet' y 'Sentiment'. Trata de buscar una forma diferente para eliminar estos campos, a la utilizada en el ejemplo anterior. 
Además, recuerda renombrar los campos 'OriginalTweet' y 'Sentiment', a 'Text' y 'Label', respectivamente. 

In [6]:
df_test = pd.read_csv(PATH+"corona/Corona_NLP_test.csv", encoding='latin-1') 
df_test.head(2)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive


In [7]:
df_test=df_test.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
df_test=df_test.rename(columns={"OriginalTweet": "Text", "Sentiment": "Label"})
df_test.head(2)

,Text,Label
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive


## Distribución de clases



In [18]:
df_train['Label'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Label, dtype: int64

In [19]:
df_test['Label'].value_counts()

Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: Label, dtype: int64

Podemos ver que el dataset contiene las siguientes 5 clases: 
- 'Extremely Negative', 
- 'Extremely Positive',
- 'Negative'
- 'Neutral'
- 'Positive'
Por tanto, estamos trabajando con un dataset para una tarea de multiclasificación de textos. 

Es recomendable transformar las clases en números, es decir, vamos a codificarlas. Por ejemplo: 

- 'Extremely Negative' -> 0
- 'Extremely Positive'-> 1
- 'Negative' -> 2
- 'Neutral' -> 3
- 'Positive' -> 4

Para ellos utilizaremos las clase LabelEncoder:

In [8]:
labels = sorted(df_train['Label'].unique().tolist())
labels

['Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral', 'Positive']

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = df_train['Label'].tolist()
y_train = le.fit_transform(y)

Vamos a verificar si las codificadiones se han realizado correctamente:

In [10]:
print(y[:15])
print(y_train[:15])

['Neutral', 'Positive', 'Positive', 'Positive', 'Extremely Negative', 'Positive', 'Positive', 'Neutral', 'Positive', 'Negative', 'Neutral', 'Extremely Positive', 'Extremely Positive', 'Positive', 'Positive']
[3 4 4 4 0 4 4 3 4 2 3 1 1 4 4]


###Ejercicio:

Ahora debes codificar también las etiquetas en el conjunto de datos del tests:

In [11]:
y_test = le.transform(df_test['Label'].tolist())
print(df_test['Label'][:15].tolist())
print(y_test[:15])

['Extremely Negative', 'Positive', 'Extremely Positive', 'Negative', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Extremely Negative', 'Extremely Positive', 'Positive', 'Extremely Negative', 'Extremely Negative', 'Extremely Positive', 'Positive']
[0 4 1 2 3 3 4 3 0 1 4 0 0 1 4]


## Construyendo un conjunto de datos para validación

El dataset únicamente proporciona un fichero para el entrenamiento y otro para test. 
Para entrenar un modelo de aprendizaje profundo y poder ajustar sus parámetros, necesitamos un conjunto de validación. Para obtenerlo podemos usar el método 'train_test_split' de la librería sklearn. 

En concreto, vamos a tomar el 20% del conjunto de entrenamiento para obtener el conjunto de validación: 

In [12]:
from sklearn.model_selection import train_test_split

X=df_train['Text'].tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y_train, 
        test_size=0.20, random_state=42, stratify=y_train)  #with random_state, we always obtain the same partition

print("instances for training: ", len(X_train))
assert len(X_train) == len(y_train)
print("instances for validation: ", len(X_val))
assert len(X_val) == len(y_val)


instances for training:  32925
instances for validation:  8232


Vamos a mostrar varios ejemplos:

In [13]:
print("Text: ", X_train[2], " Label: ", y_train[2]) 
# it must always be: 
# Tweet:  Watch this if you are one of those idiots who is panic buying food. #thinkofothers #Coronavirus : Nurse's despair as panic-buyers clear shelves https://t.co/L9aNBZjteN  Label:  0

print("\nText: ", X_val[0], " Label: ", y_val[0]) 
# it must always be: 
#Tweet:  Meanwhile a villager of quenching her thirsty in a supermarket somewhere in Africa  19 naka g  Label:  3


Text:  Watch this if you are one of those idiots who is panic buying food. #thinkofothers #Coronavirus : Nurse's despair as panic-buyers clear shelves https://t.co/L9aNBZjteN  Label:  0

Text:  Meanwhile a villager of quenching her thirsty in a supermarket somewhere in Africa  19 naka g  Label:  3
